In [205]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### assessments.csv — this file contains information about scores in tests. Typically each subject includes a series of grades followed by a final exam.

- code_module — module identification code.
- code_presentation — semester identification code.
- id_assessment — assessment identification number.
- assessment_type — test type. There are three types of assessment:
    teacher assessment (TMA), computer-aided assessment (CMA), course exam (Exam).
- date — information about the final date of the test. 
    Calculated as the number of days since the beginning of the semester. 
    The start date of the semester is numbered 0 (zero).
- weight — assessment weight in %. Usually exams are considered separately and have a weight of 100%;
    the sum of all other scores is 100%.
    
#### courses.csv — the file contains a list of all available modules (courses) and their presentations.

- code_module - module identification code.
- code_presentation — semester identification code.
- module_presentation_length — semester length in days.

#### studentAssessment.csv — this file contains student test scores. If the student does not pass (does not turn in the work, does not send the result) test, the result is not written to the table. Final exams are not accepted, if the result of the pretests is missing from the system.

- id_assessment — assessment identification number.
- id_student — student identification number.
- date_submitted — the date the student submitted the assessment, measured as the number of days since the beginning of the semester.
- is_banked — the fact of passing the test in the last semester.
- score — the student's score on this test. The range is 0 to 100. A score below 40 is interpreted as a failure.

#### studentRegistration.csv — this file contains time information, when a student has registered for an intra-semester course.

- code_module - module identification code.
- code_presentation — semester identification code.
- id_student — student identification number.
- date_registration — date of registration of the student, this is the number of days,
- measured from the beginning of the semester (for example, a negative value of -30 means that the student registered for the course 30 days before the start of the course).
- date_unregistration — the date on which the student's enrollment in the module was canceled. For students who have completed the course, this field is left blank.

In [420]:
a = pd.read_csv('data/assessments.csv')
c = pd.read_csv('data/courses.csv')
st_a = pd.read_csv('data/studentAssessment.csv')
st_reg = pd.read_csv('data/studentRegistration.csv')

#### 1. How many students have successfully passed only one course? 
#### ("Successful passing" is having a successfully passed course exam)

To know how many students successfully passed only ONE course, we should take all the assessments that have "Exam" type and check which assessments of this type got more than 40 as a score. (A score below 40 is interpreted as a failure.)

In [421]:
# We may join the a and st_a dataframes on the id_assessment column. 
# Thus we will know which score each assessment got.

st_a_joined = st_a.join(a.set_index('id_assessment'), on='id_assessment')
st_a_joined

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0


In [422]:
# From all the assessments we take only those of the type "Exam" that were successfully (score > 40) passed
# then group by id_student and aggregate, counting number of courses passed by each student,
# take the students whose number of courses passed is only 1
# and count total number of such students by taking the 0 indexed element from the .shape tuple.

one_course_succeeded = st_a_joined.query('assessment_type == "Exam" & score >= 40') \
                                .groupby('id_student') \
                                .date_submitted.agg(['count']) \
                                .rename(columns={'count': 'number_of_courses'}) \
                                .query('number_of_courses == 1') \
                                .shape[0]
print('ANSWER: {} students successfully passed only one course'.format(one_course_succeeded))

ANSWER: 3802 students successfully passed only one course


#### 2. Identify the hardest and easiest exams: find courses and exams within a course, which have the lowest and highest completion rates.

Completion = number of successfully passed exams / number of all attempts to pass the exam

In [423]:
# To solve this problem we should choose from the dataframe, joined in the first part of the task,
# assessments of type "Exam", group them by modules, semesters and id's,
# then count how many submissions of each exam took place.

exams_grouped = st_a_joined \
        .query('assessment_type == "Exam"') \
        .groupby(['code_module', 'code_presentation', 'id_assessment', 'assessment_type'], as_index=False) \
        .date_submitted.agg(['count']) \
        .rename(columns={'count': 'total_submitted'}) \
        .reset_index()
exams_grouped

,code_module,code_presentation,id_assessment,assessment_type,total_submitted
0,CCC,2014B,24290,Exam,747
1,CCC,2014J,24299,Exam,1168
2,DDD,2013B,25340,Exam,602
3,DDD,2013J,25354,Exam,968
4,DDD,2014B,25361,Exam,524
5,DDD,2014J,25368,Exam,950


In [424]:
# We should also count the number of passed exams, using the same joined dataframe as the source of data,
# remembering that to pass an exam a student should score at least 40.

exams_passed = st_a_joined[st_a_joined['score'] >= 40] \
                .query('assessment_type == "Exam"') \
                .groupby('id_assessment') \
                .date_submitted.agg(['count']).rename(columns={'count': 'passed'}).reset_index()
exams_passed

,id_assessment,passed
0,24290,664
1,24299,1019
2,25340,504
3,25354,878
4,25361,485
5,25368,842


In [425]:
# Now we join two resulting dataframes to have 'total_submitted' and 'passed' columns in one dataframe

exams_results = exams_grouped.join(exams_passed.set_index('id_assessment'), on='id_assessment')
exams_results

,code_module,code_presentation,id_assessment,assessment_type,total_submitted,passed
0,CCC,2014B,24290,Exam,747,664
1,CCC,2014J,24299,Exam,1168,1019
2,DDD,2013B,25340,Exam,602,504
3,DDD,2013J,25354,Exam,968,878
4,DDD,2014B,25361,Exam,524,485
5,DDD,2014J,25368,Exam,950,842


In [426]:
# We calculate completion rate by dividing the number of passed exams by the number of total exam submissions.

exams_results['completion_rate'] = round(exams_results['passed'] / exams_results['total_submitted'], 4)
exams_results

,code_module,code_presentation,id_assessment,assessment_type,total_submitted,passed,completion_rate
0,CCC,2014B,24290,Exam,747,664,0.8889
1,CCC,2014J,24299,Exam,1168,1019,0.8724
2,DDD,2013B,25340,Exam,602,504,0.8372
3,DDD,2013J,25354,Exam,968,878,0.9070
4,DDD,2014B,25361,Exam,524,485,0.9256
5,DDD,2014J,25368,Exam,950,842,0.8863


In [427]:
# To identify the easiest exam we take the 'id_assessment' with the highest 'completion_rate'.
# To add more information in the final output we extract the module, semester and the completion rate of the easiest exam.

easy_exam = exams_results.set_index('id_assessment')['completion_rate'].idxmax(axis=1)
easy_module = exams_results.query('id_assessment == @easy_exam')['code_module'].astype('string').values[0]
easy_semester = exams_results.query('id_assessment == @easy_exam')['code_presentation'].astype('string').values[0]
easy_completion_rate = exams_results.query('id_assessment == @easy_exam')['completion_rate'].values[0]

In [428]:
# Print formatted output

print('The easiest exam was the one with id {}, \
as a part of the module {} in the semester {}. \
Its completion rate was {}' \
.format(easy_exam, easy_module, easy_semester, easy_completion_rate))

The easiest exam was the one with id 25361, as a part of the module DDD in the semester 2014B. Its completion rate was 0.9256


In [429]:
# To identify the hardest exam we take the 'id_assessment' with the lowest 'completion_rate'.
# To add more information in the final output we extract the module, semester and the completion rate of the hardest exam.

hard_exam = exams_results.set_index('id_assessment')['completion_rate'].idxmin(axis=1)
hard_module = exams_results.query('id_assessment == @hard_exam')['code_module'].astype('string').values[0]
hard_semester = exams_results.query('id_assessment == @hard_exam')['code_presentation'].astype('string').values[0]
hard_completion_rate = exams_results.query('id_assessment == @hard_exam')['completion_rate'].values[0]

In [430]:
print('The hardest exam was the one with id {}, \
as a part of the module {} in the semester {}. \
Its completion rate was {}' \
.format(hard_exam, hard_module, hard_semester, hard_completion_rate))

The hardest exam was the one with id 25340, as a part of the module DDD in the semester 2013B. Its completion rate was 0.8372


#### 3. For each module, determine the average time for passing exams (by passing we mean the last successful passing of the exam by a student).

In [431]:
# To determine the average time for passing exams successfully, we take the assessments of type "Exam"
# with a score of at least 40 and aggregate its data by the 'date_submitted' column mean.
# date_submitted — the date the student submitted the assessment, 
# measured as the number of days since the beginning of the semester.

# We also round the mean, as such values as days are better interpreted being integers.

submission_term = st_a_joined.query('assessment_type == "Exam" & score >= 40') \
            .groupby(['code_module', 'code_presentation', 'id_assessment', 'assessment_type'], as_index=False) \
            .agg({'date_submitted': 'mean'}) \
            .rename(columns={'date_submitted': 'mean_submission_term'})
submission_term['mean_submission_term'] = round(submission_term['mean_submission_term']).astype('int32')
submission_term

,code_module,code_presentation,id_assessment,assessment_type,mean_submission_term
0,CCC,2014B,24290,Exam,232
1,CCC,2014J,24299,Exam,244
2,DDD,2013B,25340,Exam,230
3,DDD,2013J,25354,Exam,240
4,DDD,2014B,25361,Exam,235
5,DDD,2014J,25368,Exam,243


In [432]:
# To make the formatted output we use a loop, limited by the length of the submission_term dataframe.

for i in range(submission_term.shape[0]):
    module = submission_term.loc[i]['code_module']
    presentation = submission_term.loc[i]['code_presentation']
    term = submission_term.loc[i]['mean_submission_term']
    print('The mean exam submission term for module {} in the {} semester was {} days'.format(module, presentation, term))

The mean exam submission term for module CCC in the 2014B semester was 232 days
The mean exam submission term for module CCC in the 2014J semester was 244 days
The mean exam submission term for module DDD in the 2013B semester was 230 days
The mean exam submission term for module DDD in the 2013J semester was 240 days
The mean exam submission term for module DDD in the 2014B semester was 235 days
The mean exam submission term for module DDD in the 2014J semester was 243 days


#### 4. Identify the most popular courses (TOP-3) by the number of registrations for them and courses with the largest outflow (TOP-3).

In [433]:
# To identify the most popular courses by the number of registrations 
# we may take the mean number of registrations for each semester to avoid
# possible errors (counting the same students in different semesters) 

# First we should also clear the dataframe st_reg from NAN data in the 'date_registration' column
# as well as possible duplicate students in each semester.

st_reg_no_nan = st_reg.dropna(subset=['date_registration'])
st_reg_cleared = st_reg_no_nan.drop_duplicates(subset=['code_module', 'code_presentation', 'id_student'])

# Now we group the data by module and semester, then count the number of students on each semester of each module
# and count the mean number of students attending each semester for each module.
# After doing all the calculations we output top-3 courses by their popularity.

st_reg_top_3 = st_reg_cleared.groupby(['code_module', 'code_presentation'], as_index=False) \
        .id_student.agg(['count']) \
        .rename(columns={'count': 'number_of_registrations'}) \
        .reset_index() \
        .groupby('code_module', as_index=False) \
        .agg({'number_of_registrations': 'mean'}) \
        .sort_values('number_of_registrations', ascending=False) \
        .round() \
        .reset_index(drop=True) \
        .head(3)
st_reg_top_3

,code_module,number_of_registrations
0,CCC,2213.0
1,BBB,1975.0
2,FFF,1938.0


In [434]:
# Now we make a formatted output.

module_1 = st_reg_top_3.loc[0]['code_module']
module_2 = st_reg_top_3.loc[1]['code_module']
module_3 = st_reg_top_3.loc[2]['code_module']
number_1 = st_reg_top_3.loc[0]['number_of_registrations'].astype('int32')
number_2 = st_reg_top_3.loc[1]['number_of_registrations'].astype('int32')
number_3 = st_reg_top_3.loc[2]['number_of_registrations'].astype('int32')

print('ANSWER: The top-3 most popular courses are {}, {} and {}. \
\nEach of their semesters was attended on average by {}, {} and {} students respectively' \
     .format(module_1, module_2, module_3, number_1, number_2, number_3))

ANSWER: The top-3 most popular courses are CCC, BBB and FFF. 
Each of their semesters was attended on average by 2213, 1975 and 1938 students respectively


In [435]:
# To find the course with the largest outflow (number of unregistered students to the total number of registered ones)
# we may begin with preparing a dataframe grouped by 'code_module' column, 
# finding the sum of all the cancelled registrations

# At the very beginning we drop all the rows that do not include information on the course cancellations.

st_unreg_only = st_reg.dropna(subset=['date_unregistration']) \
                            .groupby(['code_module', 'code_presentation'], as_index=False) \
                            .agg({'date_unregistration': 'count'}) \
                            .groupby('code_module', as_index=False) \
                            .agg({'date_unregistration': 'sum'})
st_unreg_only

,code_module,date_unregistration
0,AAA,126
1,BBB,2377
2,CCC,1947
3,DDD,2235
4,EEE,718
5,FFF,2380
6,GGG,289


In [439]:
# Now we do the same to find the total number of registrations by course
# making a dataframe of the same size.

st_reg_only = st_reg.groupby(['code_module', 'code_presentation'], as_index=False) \
                    .agg({'date_registration': 'count'}, dropna=True) \
                    .groupby('code_module', as_index=False) \
                    .agg({'date_registration': 'sum'})
st_reg_only

,code_module,date_registration
0,AAA,748
1,BBB,7900
2,CCC,4426
3,DDD,6257
4,EEE,2932
5,FFF,7751
6,GGG,2534


In [440]:
# We join the two resulting dataframes and add a column, specifying the cancellation percentage for each course

st_reg_all = st_reg_only.join(st_unreg_only.set_index(['code_module']), on=['code_module']) \
                        .rename(columns={'date_registration': 'students_registered', 'date_unregistration': 'students_unregistered'})

st_reg_all['unreg_percetage'] = round(st_reg_all['students_unregistered'] / st_reg_all['students_registered'], 2)

st_reg_all

,code_module,students_registered,students_unregistered,unreg_percetage
0,AAA,748,126,0.17
1,BBB,7900,2377,0.30
2,CCC,4426,1947,0.44
3,DDD,6257,2235,0.36
4,EEE,2932,718,0.24
5,FFF,7751,2380,0.31
6,GGG,2534,289,0.11


In [441]:
# Now we sort the data to find the top-3 courses by the percentage of cancellations.

st_reg_all = st_reg_all.sort_values('unreg_percetage', ascending=False) \
                        .reset_index(drop=True)

st_unreg_top_3 = st_reg_all.head(3)
st_unreg_top_3

,code_module,students_registered,students_unregistered,unreg_percetage
0,CCC,4426,1947,0.44
1,DDD,6257,2235,0.36
2,FFF,7751,2380,0.31


In [442]:
# Preparing a formatted output

module_1 = st_unreg_top_3.loc[0]['code_module']
module_2 = st_unreg_top_3.loc[1]['code_module']
module_3 = st_unreg_top_3.loc[2]['code_module']

percentage_1 = st_unreg_top_3.loc[0]['unreg_percetage']
percentage_2 = st_unreg_top_3.loc[1]['unreg_percetage']
percentage_3 = st_unreg_top_3.loc[2]['unreg_percetage']

print('ANSWER: The courses with the biggest outflow of students are {}, {} and {}. \n{:,.0%}, {:,.0%} and {:,.0%} of students \
unregistered from these courses respectively'.format(module_1, module_2, module_3, percentage_1, percentage_2, percentage_3))

ANSWER: The courses with the biggest outflow of students are CCC, DDD and FFF. 
44%, 36% and 31% of students unregistered from these courses respectively


#### 5. Write a python function that allows you to build a cohort (semester) analysis. Between the beginning of 2013 and the end of 2014, identify the semester with the lowest course completion rates and the longest average course completion times.

In [544]:
# To solve this problem we may create a cohort table, that would include all the students with their
# course final scores and terms of final tasks submission. 

# First we join the st_a and a dataframes and calculate a new column 'weighed_score'
# to know, how much every student got for each task, 
# keeping in mind, that there are some courses with no exams, 
# where a student must score at least 40 to pass.

cohorts = st_a.join(a.set_index('id_assessment'), on='id_assessment')
cohorts['weighed_score'] = cohorts['score'] * cohorts['weight'] * 0.01
cohorts

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,weighed_score
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,7.8
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,7.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,7.2
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,6.9
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,7.9
...,...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0,0.0


In [545]:
# From the past analysis we know, that only 'CCC' and 'DDD' courses have exams. 
# Thus we calculate the total weighed score in every not-exam-type course for each student.

cohorts_no_exam = cohorts.query('code_module in ["AAA", "BBB", "EEE", "FFF", "GGG"]') \
                                        .groupby(['code_module', 'code_presentation', 'id_student'], as_index=False) \
                                        .agg({'weighed_score': 'sum'})
cohorts_no_exam

,code_module,code_presentation,id_student,weighed_score
0,AAA,2013J,11391,82.4
1,AAA,2013J,28400,65.4
2,AAA,2013J,31604,76.3
3,AAA,2013J,32885,55.0
4,AAA,2013J,38053,66.9
...,...,...,...,...
17481,GGG,2014J,2620947,0.0
17482,GGG,2014J,2645731,0.0
17483,GGG,2014J,2648187,0.0
17484,GGG,2014J,2679821,0.0


Interestingly we can see that many students in the 'GGG' course had a weighed score of 0.
Let's group the dataframe by 'code_module' to check if there is some logic in it

In [546]:
cohorts_no_exam.groupby('code_module').agg({'weighed_score': 'mean'})

,weighed_score
code_module,
AAA,60.379433
BBB,52.612174
EEE,67.838190
FFF,55.950191
GGG,0.000000


Now we can see that the 'GGG' course tasks had **no weight at all** (or there is a mistake in the data). Anyway we don't need this module in our calculation no more, so it's going to be dropped from the final cohort dataframe.

In [547]:
# Now we calculate the students' score in every exam-type course,
# choosing only the rows that refer to the exam result ('assessment_type' == "Exam"),
# because a student is considered to pass course successfully if they pass the exam.

cohorts_exam = cohorts.query('code_module in ["CCC", "DDD"] & assessment_type == "Exam"') \
                                        .groupby(['code_module', 'code_presentation', 'id_student'], as_index=False) \
                                        .agg({'weighed_score': 'sum'})
cohorts_exam

,code_module,code_presentation,id_student,weighed_score
0,CCC,2014B,29764,94.0
1,CCC,2014B,29820,76.0
2,CCC,2014B,40604,66.0
3,CCC,2014B,42638,50.0
4,CCC,2014B,46605,98.0
...,...,...,...,...
4954,DDD,2014J,2677210,71.0
4955,DDD,2014J,2678380,64.0
4956,DDD,2014J,2684043,69.0
4957,DDD,2014J,2685863,49.0


In [568]:
# Now we concatenate the two dataframes, where we have the each student's score in each module and semester.

cohorts_all = pd.concat([cohorts_no_exam, cohorts_exam])
cohorts_all = cohorts_all.reset_index(drop=True)
cohorts_all

,code_module,code_presentation,id_student,weighed_score
0,AAA,2013J,11391,82.4
1,AAA,2013J,28400,65.4
2,AAA,2013J,31604,76.3
3,AAA,2013J,32885,55.0
4,AAA,2013J,38053,66.9
...,...,...,...,...
22440,DDD,2014J,2677210,71.0
22441,DDD,2014J,2678380,64.0
22442,DDD,2014J,2684043,69.0
22443,DDD,2014J,2685863,49.0


In [549]:
# Let's check what happens to the 'date_submitted' column if the student's score is taken from the previous semester

cohorts.query('is_banked == 1')

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,weighed_score
1641,1758,603861,-1,1,61.0,AAA,2014J,TMA,19.0,10.0,6.1
1685,1758,1970390,-1,1,78.0,AAA,2014J,TMA,19.0,10.0,7.8
1690,1758,2073016,-1,1,60.0,AAA,2014J,TMA,19.0,10.0,6.0
1693,1758,2139353,-1,1,65.0,AAA,2014J,TMA,19.0,10.0,6.5
1708,1758,2358969,-1,1,68.0,AAA,2014J,TMA,19.0,10.0,6.8
...,...,...,...,...,...,...,...,...,...,...,...
172516,37440,624767,-1,1,100.0,GGG,2014J,CMA,229.0,0.0,0.0
172526,37440,631446,-1,1,100.0,GGG,2014J,CMA,229.0,0.0,0.0
172717,37441,624767,-1,1,100.0,GGG,2014J,CMA,229.0,0.0,0.0
172727,37441,631446,-1,1,100.0,GGG,2014J,CMA,229.0,0.0,0.0


In [557]:
# We see that the 'date_submitted' value turns into -1, so all these rows must be dropped from the dataframe
# for further analysis of submission terms.

terms = cohorts
terms.drop(terms.loc[terms['is_banked'] == 1].index, inplace=True)
terms

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight,weighed_score
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0,7.8
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0,7.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0,7.2
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0,6.9
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0,7.9
...,...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0,0.0


In [555]:
# We take the terms dataframe and calculate first the mean submission term for each 'id_assessment'

terms.groupby(['code_module', 'code_presentation', 'id_assessment']) \
                .agg({'date_submitted': 'mean'}) \
                .head(30)

date_submitted
code_module code_presentation id_assessment                
AAA         2013J             1752                19.356546
                              1753                55.959064
                              1754               117.235650
                              1755               167.303630
                              1756               214.543624
            2014J             1758                19.700000
                              1759                56.360656
                              1760               119.108475
                              1761               168.741007
                              1762               214.992754
BBB         2013B             14984               18.549516
                              14985               48.320692
                              14986               89.767857
                              14987              125.584318
                              14988              159.973744
                              14989              186.165796
                              14991               60.000000
                              14992               95.000939
                              14993              126.077472
                              14994              164.034261
                              14995              189.003448
            2013J             14996               17.545897
                              14997               47.527665
                              14998               96.353535
                              14999              131.542904
                              15000              167.240484
                              15001              206.472357
                              15003               56.000000
                              15004              102.000000
                              15005              133.004055

We may see that there is always a maximum value of the submission term ('date_submitted' column), which is the date when the last assessment is submitted, thus ending a course for a student.
Therefore, we should take **the maximum value of submission term for each student** for further cohort calculations. 

In [558]:
terms = terms.groupby(['code_module', 'code_presentation', 'id_assessment', 'id_student'], as_index=False) \
                .agg({'date_submitted': 'mean'}) \
                .groupby(['code_module', 'code_presentation', 'id_student'], as_index=False) \
                .agg({'date_submitted': 'max'})
terms

,code_module,code_presentation,id_student,date_submitted
0,AAA,2013J,11391,212
1,AAA,2013J,28400,212
2,AAA,2013J,31604,213
3,AAA,2013J,32885,222
4,AAA,2013J,38053,215
...,...,...,...,...
25575,GGG,2014J,2620947,222
25576,GGG,2014J,2645731,216
25577,GGG,2014J,2648187,215
25578,GGG,2014J,2679821,75


In [569]:
# Now we join the terms dataframe to have a corresponding 'date_submitted' value for each student in cohorts

cohorts_all = cohorts_all.join(terms.set_index(['code_module', 'code_presentation', 'id_student']), \
                                             on=['code_module', 'code_presentation', 'id_student'], how='left')


In [573]:
cohorts_all

,code_module,code_presentation,id_student,weighed_score,date_submitted
0,AAA,2013J,11391,82.4,212.0
1,AAA,2013J,28400,65.4,212.0
2,AAA,2013J,31604,76.3,213.0
3,AAA,2013J,32885,55.0,222.0
4,AAA,2013J,38053,66.9,215.0
...,...,...,...,...,...
22440,DDD,2014J,2677210,71.0,243.0
22441,DDD,2014J,2678380,64.0,242.0
22442,DDD,2014J,2684043,69.0,243.0
22443,DDD,2014J,2685863,49.0,243.0


In [577]:
# We can now calculate the mean submission term for each semester among the students that passed the courses. 
# We may include the 'GGG' module for this calculation, because, not being rated with scores it still has
# values of submission term.

submission_terms = cohorts_all.query('weighed_score >= 40 | code_module == "GGG"') \
                                .groupby(['code_presentation']) \
                                .agg({'date_submitted': 'mean'})['date_submitted']
submission_terms

code_presentation
2013B    202.956089
2013J    206.299421
2014B    202.397225
2014J    212.897216
Name: date_submitted, dtype: float64

In [581]:
# To avoid errors with calculation of success percentage scores by semester we should divide the number of students who passed
# courses except for 'GGG' course by the total number of students who were not part of 'GGG' course.

success_percentage = cohorts_all.query('weighed_score >= 40 & code_module != "GGG"') \
            .groupby('code_presentation') \
            .agg({'id_student': 'count'})['id_student'] / cohorts_all \
            .query('code_module != "GGG"') \
            .groupby('code_presentation') \
            .agg({'id_student': 'count'})['id_student']
success_percentage

code_presentation
2013B    0.722322
2013J    0.762535
2014B    0.760550
2014J    0.780339
Name: id_student, dtype: float64

In [583]:
worst_semester = success_percentage.idxmin()
worst_rate = success_percentage.min()

print('ANSWER: The semester with the lowest percentage of students who passed a course was {}, when the success rate was only {:,.0%}' \
      .format(worst_semester, worst_rate))

ANSWER: The semester with the lowest percentage of students who passed a course was 2013B, when the success rate was only 72%


In [587]:
longest_semester = submission_terms.idxmax()
longest_term = round(submission_terms.max())

print('ANSWER: The semester with the longest course submission terms was {}, \
when the mean final submission term reached {} days' \
      .format(longest_semester, longest_term))

ANSWER: The semester with the longest course submission terms was 2014J, when the mean final submission term reached 213 days
